In [72]:
%matplotlib inline
from __future__ import print_function
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
#from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
from scipy.stats import sem
from sklearn.preprocessing import PolynomialFeatures
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2

# Contains linear models, e.g., linear regression, ridge regression, LASSO, etc.
import sklearn.linear_model as sklin
# Provides train-test split, cross-validation, etc.
import sklearn.cross_validation as skcv
# Provides grid search functionality
import sklearn.grid_search as skgs
# For data normalization
import sklearn.preprocessing as skpr
# Allows us to create custom scoring functions
import sklearn.metrics as skmet
from sklearn import grid_search

In [2]:
def eval_pred(predictor, X, y, K=3):
    #scores = cross_val_score(pred, X, y, cv = 3)
    scores = skcv.cross_val_score(predictor, X, y, scoring='accuracy', cv=5)
    print('C-V score =', np.mean(scores), '+/-', np.std(scores))

In [3]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [4]:
def score(gtruth, pred):
    diff = gtruth - pred
    return np.sqrt(np.mean(np.square(diff)))

In [5]:
X = pd.read_csv('train.csv')
del X['Id']
y = X.y
del X['y']
pd.DataFrame(X).head()
X_test = pd.read_csv('test.csv')
del X_test['Id']

print(X.shape, y.shape, X_test.shape)


(1000, 15) (1000,) (3000, 15)


In [410]:
#normalise the data
from sklearn.feature_selection import VarianceThreshold
scaler= skpr.StandardScaler(with_mean=True, with_std=True, copy=True)
X_scaled = scaler.fit_transform(X)
# build pipeline to train
a = [100]
polynomial_features = PolynomialFeatures(include_bias=True, interaction_only=False, degree = 2)

svm = SVC(C=1.0, kernel='rbf', gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, 
         class_weight=None, verbose=True, max_iter=-1, decision_function_shape='ovr', random_state=None, degree = 2)

sfm = SelectFromModel(svm, prefit=False)
#25 0.929
#5 0.927
#70 0.924
#66 0.927
thres_value = 30
selector = VarianceThreshold(thres_value)
X_new = selector.fit_transform(X)
X_new_scaled = scaler.fit_transform(X_new)
estimators = Pipeline([('pf', polynomial_features),
                             #('feature_selection', sfm),
                       ('svm', svm)])
parameters = {'svm__C':
              [1,2,3,4,5],
              #np.logspace(-3,3,7),
              #'svm__degree':[2,3,4],
              #'feature_selection__threshold':np.linspace(0.3,0.9,8),
              'pf__interaction_only':[False], 'svm__gamma':
              #[0.0085]
              np.linspace(0.015,0.017,15),
              'svm__degree':
              [2]
              #[]
              ,}

clf = grid_search.GridSearchCV(estimators, param_grid=parameters, n_jobs=-1, verbose=1)
clf.fit(X_new_scaled,y)

print("Best score: %0.3f" % clf.best_score_)
print("Best params:",  clf.best_params_)
print("value of variance threshold", thres_value)
print("shape of X_new_scaled", X_new_scaled.shape)

#eval_pred(clf, X, y, K=3)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    2.2s


[LibSVM]Best score: 0.932
Best params: {'svm__C': 5, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016142857142857143}
value of variance threshold 30
shape of X_new_scaled (1000, 7)


[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:    3.1s finished


In [411]:
# normalise X_test data

clf.grid_scores_.sort(key=lambda x: x.mean_validation_score,
    reverse=True)
print(clf.grid_scores_[0:10])

thres_value = 30
selector = VarianceThreshold(thres_value)
X_test_new = selector.fit_transform(X_test)
X_test_new_scaled = scaler.fit_transform(X_test_new)
X_test_scaled = X_test_new_scaled
print("shape of X_test_scaled", X_test_scaled.shape)
print (clf.predict(X_test_scaled))

[mean: 0.93200, std: 0.00997, params: {'svm__C': 5, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016142857142857143}, mean: 0.93200, std: 0.00997, params: {'svm__C': 5, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016285714285714285}, mean: 0.93200, std: 0.00997, params: {'svm__C': 5, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016428571428571428}, mean: 0.93200, std: 0.00997, params: {'svm__C': 5, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016571428571428574}, mean: 0.93100, std: 0.00890, params: {'svm__C': 4, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016714285714285716}, mean: 0.93100, std: 0.00890, params: {'svm__C': 4, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.016857142857142859}, mean: 0.93100, std: 0.00890, params: {'svm__C': 4, 'pf__interaction_only': False, 'svm__degree': 2, 'svm__gamma': 0.017000000000000001}, mean: 0.93100, std: 0.00890, params: {'s

In [412]:
sub = pd.read_csv("sample.csv")
sub['y'] = clf.predict(X_test_scaled)
sub.head()
sub.to_csv('test_basic.csv', index = False)

In [300]:
selector = VarianceThreshold(66)

In [301]:
X_varth = selector.fit_transform(X)

In [302]:
X_varth.shape

(1000, 6)

In [80]:
X_pos.shape
X_new = SelectKBest(chi2, k=2).fit_transform(X_pos, y)

(1000, 15)

# This is just useless test

In [118]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#neigh.fit(X, y) 

estimators = Pipeline([('ng', neigh)])
parameters = {'ng__n_neighbors':[5,6,7,8,9,10,11,12,13,14,15] }

clf = grid_search.GridSearchCV(estimators, param_grid=parameters, n_jobs=-1, verbose=1)
clf.fit(X_scaled,y)

print("Best score: %0.3f" % clf.best_score_)
print("Best params:",  clf.best_params_)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
Best score: 0.889
Best params: {'ng__n_neighbors': 11}


[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    0.2s finished
